# This section uses Magpie to calculate surface brightness profile

This is equivalently what the previous section does, but magpie is used to calculate surface brightness profile for each Ha Hb line maps and weights, averaged out over the polar angle phi, for each given pixel distance.

what does the package [magpie](https://github.com/knaidoo29/magpie/tree/master) does:

It transforms grid in Cartesian coordinates into (r,phi) polar coordinates, while preserving the size of the surface area of each pixel. Therefore one can calculate surface brightness of a certain ring area by summing over phi of a given radius range. This method is used in [Matharu 2023](https://iopscience.iop.org/article/10.3847/2041-8213/acd1db/pdf) and [Matharu 2024](https://arxiv.org/pdf/2404.17629).

In [1]:
import  magpie              as     magpie

In [2]:

import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
from    concurrent.futures  import ThreadPoolExecutor, as_completed
import  sys, os
from    IPython.display     import clear_output



#this is just a handy little function to return the desired file path
#give one entry in the object list, return the desired file path
def file_name(obj,prefix,filetype='fits'):
    field = obj['subfield'].lower()
    id    = str(obj['ID']).zfill(5)
    return f"hlsp_clear_hst_wfc3_{field}-{id}_g102-g141_v4_{prefix}.{filetype}"

def find_data(name,hdu):
    for image in hdu:
        if name == image.name:
            return image

def save_update(image_to_save,extracted):
        for i,image in enumerate(extracted):
            if image.name == image_to_save.name:
                extracted[i] = image_to_save
                return extracted
        extracted.append(image_to_save)
        extracted.flush()
        return extracted

def errorcounting(results):
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

obj_lis = Table.read('obj_lis_selected.fits')

### calculate radial profile

In [3]:
import magpie.montecarlo

def spatial_remap(map,pixel_length):
    b2r = magpie.montecarlo.Box2Ring()
    b2r.setup_box(-25*pixel_length,25*pixel_length,50,
                -25*pixel_length,25*pixel_length,50)
    b2r.setup_polar_lin(0., 25*pixel_length, 12, 10, center=[0., 0.])
    b2r.get_weights() 
    return np.linspace(b2r.redges[0], b2r.redges[-1],12), b2r.remap(map)

def radial_profile(obj,linemap,weight,seg,pixel_length):
        linemap = linemap.data; weight = weight.data; seg = seg.data

        seg = np.where(seg==obj['ID'],1,0)
        seg = np.where(linemap>0,seg,0)
        seg = spatial_remap(seg,pixel_length)[1]
        seg_weight = seg/np.sum(seg,axis=0)

        r, linemap_r,   = spatial_remap(np.where(linemap>0,linemap,0),pixel_length)
        linemap_r_wht = spatial_remap(np.where(linemap>0,weight,0),pixel_length)[1]


        map_r   = np.average(linemap_r,weights=seg_weight,axis=0)
        map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
        map_r_std = np.average((linemap_r-map_r)**2,weights = seg_weight,axis=0)**0.5
        map_r_err = (map_r_err**2 + map_r_std**2)**0.5
        return r, map_r, map_r_err 


def gen_radial_table(obj,LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                    LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV'):
    #try:
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        
        r, ha_r, ha_r_err = radial_profile(obj,
                                        linemap      = find_data(LINE_HA,hdu),
                                        weight       = find_data(LINEWHT_HA,hdu),
                                        seg          = find_data('SEG',hdu),
                                        pixel_length = obj['pixel_length'])
        
        r, hb_r, hb_r_err = radial_profile(obj,
                                        linemap      = find_data(LINE_HB,hdu),
                                        weight       = find_data(LINEWHT_HB,hdu),
                                        seg          = find_data('SEG',hdu),
                                        pixel_length = obj['pixel_length'])

        balmer_r     = ha_r/hb_r
        balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
        
        cols = [
            fits.Column(name='DISTANCE [kpc]',                       format='E', array=r),
            fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=ha_r),
            fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
            fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=hb_r),
            fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
            fits.Column(name='BALMER_DECREM',                        format='E', array=balmer_r),
            fits.Column(name='BALMER_DECREM_ERR',                    format='E', array=balmer_r_err),
        ]
        
        #choose the right name for saving the radial table
        if 'CONV' in LINE_HB:
            name_addon = '_CONV'
        else:
            name_addon = ''
        if 'BG' not in LINE_HA:
            name = f'RAD_PROFILE{name_addon}'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)
        else:
            name = f'RAD_PROFILE{name_addon}_BG'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)

        #save or update table
        save_update(new_table,hdu)
        hdu.flush()
        return f"{obj['subfield']}-{obj['ID']} processed"
    #except Exception as e:
    #        return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"


def cat_process(obj_lis,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB,max_threads=1):
        print(f'start process,{LINE_HA},{LINE_HB}')
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(
                    gen_radial_table,
                    obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB
                                            ): obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in obj_lis:
                results.append(gen_radial_table(obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB))
            return results


def main():
    obj_lis = Table.read('obj_lis_selected.fits')
    
    results1 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=6)
    errorcounting(results1)

    results2 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=6)
    errorcounting(results2)


    results3 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=6)
    errorcounting(results3)

    
    results4 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_CONV_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=6)
    errorcounting(results4)

if __name__ == '__main__':
    main()


start process


Processing:   0%|          | 0/158 [00:00<?, ?it/s]/tmp/ipykernel_178260/4193684850.py:17: RuntimeWarning: invalid value encountered in divide
  seg_weight = seg/np.sum(seg,axis=0)
/tmp/ipykernel_178260/4193684850.py:24: RuntimeWarning: divide by zero encountered in divide
  map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
Processing: 100%|██████████| 158/158 [02:07<00:00,  1.24it/s]


total number of obj processed: 158
number of failed obj 0
start process


Processing: 100%|██████████| 158/158 [02:08<00:00,  1.23it/s]


total number of obj processed: 158
number of failed obj 0
start process


Processing: 100%|██████████| 158/158 [03:18<00:00,  1.26s/it]


total number of obj processed: 158
number of failed obj 0
start process


Processing: 100%|██████████| 158/158 [02:16<00:00,  1.16it/s]

total number of obj processed: 158
number of failed obj 0


### plot radial profiles

In [12]:
%matplotlib inline
def plot_balmer_decrem(obj,plot,plot_var,crop_size=40):

    try:
        path = f"data_extracted/{file_name(obj,prefix='extracted')}"
        with fits.open(path) as hdu:
            center_x, center_y = np.array(hdu[3].data.shape) // 2
            half_crop = crop_size // 2
                
            ax = plt.figure(figsize=(12,15));i=1
            
            for name in ['DSCI','LINE_HA','LINE_HB','LINE_HB_CONV']:
                image = find_data(name,hdu)
                data  = image.data[
                    center_y - half_crop : center_y + half_crop,
                    center_x - half_crop : center_x + half_crop]
                ax.add_subplot(int(f'32{i}'));i+=1
                plt.imshow(data,#np.where(seg,hdu[index].data,0),
                            norm=colors.Normalize(vmin=0),
                            origin='lower',
                            cmap = 'plasma_r')
                plt.plot([3,7],[4,4])
                plt.text(5,5,f"{round(obj['pixel_length']*4,2)} kpc")
                plt.title(f"{image.name}_{obj['subfield']}_{obj['ID']}{obj['tag']}")
                plt.colorbar()
            
            r,ha_r,ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err = np.vstack(find_data(plot,hdu).data).transpose()
            r,ha_r_var,ha_r_err_var, hb_r_var, hb_r_err_var, balmer_r_var, balmer_r_err_var = np.vstack(find_data(plot_var,hdu).data).transpose()
            
            ax.add_subplot(325)
            plt.errorbar(r,ha_r,yerr = ha_r_err,fmt='bo:',label=f'Ha_{plot}')
            plt.errorbar(r,hb_r,yerr = hb_r_err,fmt='go:',label=f'Hb_{plot}')
            plt.errorbar(r,ha_r_var,yerr = ha_r_err_var,fmt='bo:',label=f'Ha_{plot_var}',alpha=0.4)
            plt.errorbar(r,hb_r_var,yerr = hb_r_err_var,fmt='go:',label=f'Hb_{plot_var}',alpha=0.4)
            plt.xlabel('distance [kpc]'); plt.ylabel('flux [1e-7 erg/s/cm2]')
            plt.yscale('log');plt.grid();plt.legend()
            
            ax.add_subplot(326)
            plt.errorbar(r,balmer_r,yerr = balmer_r_err,fmt='ro:',label=f'balmer_decrem_{plot}')
            plt.errorbar(r,balmer_r_var,yerr = balmer_r_err_var,fmt='bo:',label=f'balmer_decrem_{plot_var}',alpha=0.4)
            plt.xlabel('distance [kpc]');plt.ylabel('Ha/Hb')
            plt.xlim(0,8);plt.ylim(-5,10)
            plt.grid();plt.legend()

            save_path = f"radial_balmer_decrem/{plot}_vs_{plot_var}"
            save_path_sn_10 = f"sn_10/radial_balmer_decrem/{plot}_vs_{plot_var}"
            os.makedirs(save_path,exist_ok=True)
            os.makedirs(save_path_sn_10,exist_ok=True)

            plt.savefig(f"{save_path}/{obj['subfield']}-{obj['ID']}.png")
            if obj['sn_hb'] >10:
                plt.savefig(f"{save_path_sn_10}/{obj['subfield']}-{obj['ID']}.png")
            plt.close('all')

            return f"{obj['subfield']}-{obj['ID']}saved"
        
    except Exception as e:
            return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"


def cat_process(obj_lis,plot='RAD_PROFILE',plot_var='RAD_PROFILE_BG',max_threads=1):
        print(f'\n start plot process{plot,plot_var}')
        results = []
        if max_threads>1:
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(plot_balmer_decrem,
                                            obj,plot=plot,plot_var=plot_var
                                            ):obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in tqdm(obj_lis):
                results.append(plot_balmer_decrem(obj,plot,plot_var))
            return results


def main():
    use('Agg')
    obj_lis = Table.read('obj_lis_selected.fits')
    results1 = cat_process(obj_lis,
                        plot='RAD_PROFILE',plot_var='RAD_PROFILE_BG',max_threads=1)
    errorcounting(results1)

    results2 = cat_process(obj_lis,
                        plot='RAD_PROFILE',plot_var='RAD_PROFILE_CONV',max_threads=1)
    errorcounting(results2)

    results3 = cat_process(obj_lis,
                        plot='RAD_PROFILE_BG',plot_var='RAD_PROFILE_CONV_BG',max_threads=1)
    errorcounting(results3)
    
if __name__ == '__main__':
    main()


 start plot process('RAD_PROFILE', 'RAD_PROFILE_BG')


100%|██████████| 158/158 [02:08<00:00,  1.23it/s]


total number of obj processed: 158
number of failed obj 0

 start plot process('RAD_PROFILE', 'RAD_PROFILE_CONV')


100%|██████████| 158/158 [02:03<00:00,  1.27it/s]


total number of obj processed: 158
number of failed obj 0

 start plot process('RAD_PROFILE_BG', 'RAD_PROFILE_CONV_BG')


100%|██████████| 158/158 [02:00<00:00,  1.31it/s]

total number of obj processed: 158
number of failed obj 0
